In [1]:
import tensorflow as tf
import numpy as np
from numpy import linalg as la

/Users/jackweissenberger/anaconda3/envs/fastMM/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
tf.reset_default_graph()
tf.set_random_seed(25)

In [4]:
def strass(A, B, steps):
  
    #Check Dimensions
    # tensor.get_shape().as_list()
    (m,n) = A.get_shape().as_list()
    (nn, p) = B.get_shape().as_list()

    #old code case m, n, nn, and p as ints
    
    if n != nn: raise ValueError("incompatible dimensions")
    C = tf.zeros([m,p])
    
    #Base case
    if steps == 0 or m ==1 or n ==1 or p == 1:
        C = tf.matmul(A,B)
        return C
    
    #Dynamic peeling
    if m % 2 == 1:
        #C[:m-1, :] 
        Cmat= strass(A[:m-1,:],B, steps)
        #C[m-1,:]
        guy = A[m-1,:]
        
        #delete these two
        q= guy.get_shape()
        print("(q,w):", q)
        
        
        Crow = tf.matmul(tf.expand_dims(A[m-1,:],0),B)
        return tf.concat([Cmat, Crow], 0)
    if n % 2 == 1:
        Cmat = strass(A[:, :n-1], B[:n-1,:], steps)
        C = tf.add(Cmat,  tf.matmul(tf.expand_dims(A[:,n-1],1),tf.expand_dims(B[n-1,:],0)))
        return C
    if p % 2 == 1:
        #C[:, :p-1]
        Cmat = strass(A, B[:,:p-1], steps)
        #C[:,p-1]
        Ccol = tf.matmul(A,tf.expand_dims(B[:,p-1],1))
        return tf.concat([Cmat, Ccol], 1)
    
    # divide when m, n and p are all even
    m2 = int(m/2)
    n2 = int(n/2)
    p2 = int(p/2)
    A11 = A[:m2,:n2] 
    A12 = A[:m2,n2:]
    A21 = A[m2:,:n2] 
    A22 = A[m2:,n2:]
    B11 = B[:n2,:p2]   
    B12 = B[:n2,p2:]
    B21 = B[n2:,:p2] 
    B22 = B[n2:,p2:]
    
    # conquer
    M1 = strass(A11, tf.subtract(B12,B22)   ,steps-1)
    M2 = strass(tf.add(A11,A12), B22   ,steps-1)
    M3 = strass(tf.add(A21,A22),B11    ,steps-1)
    M4 = strass(A22    ,tf.subtract(B21,B11),steps-1)
    M5 = strass(tf.add(A11, A22), tf.add(B11, B22),steps-1)
    M6 = strass( tf.subtract(A12,A22), tf.add(B21,B22),steps-1)
    M7 = strass(tf.subtract(A11,A21), tf.add(B11, B12),steps-1)
    
    # conquer    
    #C[:m2,:p2] 
    C11 = tf.add(tf.subtract(tf.add(M5, M4), M2), M6) 
    #C[:m2,p2:]
    C12 = tf.add(M1, M2) 
    #C[m2:,:p2] 
    C21 = tf.add(M3,M4)
    #C[m2:,p2:]
    C22 = tf.subtract(tf.subtract(tf.add(M1,M5), M3), M7)
    
    # nation building
    C1 = tf.concat([C11, C12], 1)
    C2 = tf.concat([C21,C22], 1)
    C = tf.concat([C1,C2], 0)
    
    return C

In [5]:
with tf.Session() as sess:
    a = np.random.rand(3,4)
    w = np.identity(4)

    a = tf.constant(a, dtype=tf.float64)
    w = tf.constant(w)
    
    m = sess.run(tf.matmul(a,w))
    s = sess.run(strass(a,w,1))
    
    print("A: \n", sess.run(a), '\n')
    print("matmul: \n", m)
    print("Strass: \n", s)
    print("\n m-s: \n", m-s, '\n')
    print("Strassen Error: ", la.norm(s-m, 'fro')/la.norm(s))


(q,w): (4,)
A: 
 [[0.92  0.43  0.802 0.587]
 [0.591 0.063 0.064 0.812]
 [0.079 0.701 0.057 0.507]] 

matmul: 
 [[0.92  0.43  0.802 0.587]
 [0.591 0.063 0.064 0.812]
 [0.079 0.701 0.057 0.507]]
Strass: 
 [[0.92  0.43  0.802 0.587]
 [0.591 0.063 0.064 0.812]
 [0.079 0.701 0.057 0.507]]

 m-s: 
 [[1.11e-16 1.11e-16 0.00e+00 0.00e+00]
 [0.00e+00 0.00e+00 1.11e-16 1.11e-16]
 [0.00e+00 0.00e+00 0.00e+00 0.00e+00]] 

Strassen Error:  1.1400525616201925e-16


In [6]:
#with tf.Session() as sess:
    a = np.random.rand(4,4)
    w = np.identity(4)

    a = tf.constant(a, dtype=tf.float64)
    w = tf.constant(w)
    
    a = sess.run(a)
    w = sess.run(w)
    
    s = strass(a,w, 2)

RuntimeError: Attempted to use a closed Session.

In [5]:
s

array([[0.914128  , 0.04061079, 0.37017392, 0.2666174 ],
       [0.68150857, 0.72203561, 0.02497239, 0.55707602],
       [0.84485791, 0.30971141, 0.5360951 , 0.19091652],
       [0.92230492, 0.078655  , 0.69188986, 0.423988  ]])